In [95]:
using QuantumOptics
using OrdinaryDiffEq
using ModelingToolkit
using LinearAlgebra
using Symbolics
using SymbolicUtils
using DifferentialEquations

In [96]:
include("src/indexing.jl")
include("src/doubleSums.jl")
include("src/averageSums.jl")
include("src/indexedMeanfield.jl")

isNotIn (generic function with 1 method)

In [97]:
N = 3
ha = NLevelSpace(:a,2)
hf= FockSpace(:field) #HilbertSpace of the Field
h = tensor(hf,ha)


a = Destroy(h,:a)

σ(i,j) = Transition(h,:σ,i,j)


j_ind = Index(h,:j,N)
k_ind = Index(h,:k,N)
l_ind = Index(h,:l,N)

Δ = 100.0
g_j = IndexedVariable(:g,j_ind)

#define some Indexed Operators
#need only to define ops in Hamiltonian and such
#if any other operator gets evaluated, it will automaticly "upgrade" it to an indexed operator
#   with index corresponding to the index, that was used for evaluation

σ12_j = IndexedOperator(σ(1,2),j_ind)
σ21_j = IndexedOperator(σ(2,1),j_ind)



σ12_k = IndexedOperator(σ(1,2),k_ind)
σ21_k = IndexedOperator(σ(2,1),k_ind)
σ22_k = IndexedOperator(σ(2,2),k_ind)

σ12_l = IndexedOperator(σ(1,2),l_ind)


#Define Sums
sum1 = IndexedSingleSum(g_j*σ12_j*a',j_ind)
sum2 = IndexedSingleSum(g_j*σ21_j*a,j_ind)

#can also be done in one step as:
sum_both = IndexedSingleSum((g_j*σ12_j*a' + g_j*σ21_j*a),j_ind)


#Hamiltonian:
H = -Δ*a'*a + sum_both

i_ind = Index(h,:i,N)
σ21_i = IndexedOperator(σ(2,1),i_ind)


σ21i

In [98]:
i_ind = Index(h,:i,N)
σ12_i = IndexedOperator(σ(1,2),i_ind)
ops = [σ22_k]
J = [σ12_i,σ12_j]
Jdagger = adjoint.(J)
rates_ind = DoubleIndexedVariable(:Γ,i_ind,j_ind)

Γij

In [99]:
#_master_lindblad(a_,J::Vector{IndexedOperator},Jdagger::Vector{IndexedOperator},rates::SymbolicUtils.Sym{Parameter,DoubleIndexedVariable})
indexed_master_lindblad(ops[1],J,Jdagger,[rates_ind])

(Σ(j=1:3)(j≠k)Σ(i=1:3)(i≠j,k)Γij*(σ21i*σ12j*σ22k)+0+Σ(j=1:3)(j≠k)Γjj*(σ22j*σ22k)+Σ(i=1:3)(i≠k)-0.5Γik*(σ21i*σ12k)+-0.5Γkk*(σ22k)+Σ(j=1:3)(j≠k)Σ(i=1:3)(i≠j,k)-0.5Γij*(σ21i*σ12j*σ22k)+Σ(j=1:3)(j≠k)-0.5Γjj*(σ22j*σ22k)+-0.5Γkk*(σ22k)+Σ(j=1:3)(j≠k)Σ(i=1:3)(i≠j,k)-0.5Γij*(σ21i*σ12j*σ22k)+Σ(j=1:3)(j≠k)-0.5Γkj*(σ12j*σ21k)+-0.5Γkk*(σ22k)+Σ(j=1:3)(j≠k)-0.5Γjj*(σ22j*σ22k))

In [100]:
#indexedMeanfield(a::Vector,H,J::Vector{IndexedOperator}
eqs = meanfield(ops,H,J;rates=[rates_ind])

∂ₜ(⟨σ22k⟩) = (0 + 1im)*gk*⟨a′*σ12k⟩ + (0 - 1im)*gk*⟨a*σ21k⟩ - 0.5var"∑(i=1:3)(i≠k)Γik*⟨σ21i*σ12k⟩" - 0.5var"∑(j=1:3)(j≠k)Γkj*⟨σ12j*σ21k⟩" - 1.5Γkk*⟨σ22k⟩


In [101]:
eqs_expanded = cumulant_expansion(eqs,2) #<- constant time (depends on number of initial equations from meanfield)
1

1

In [102]:
me_comp = indexedComplete(eqs_expanded)
1

1

In [103]:
me_comp_ = evalME(me_comp)


∂ₜ(⟨σ221⟩) = (0 + 1im)*g1*⟨a′*σ121⟩ + (0 - 1im)*g1*⟨a*σ211⟩ - 0.5Γ21*⟨σ121*σ212⟩ - 0.5Γ31*⟨σ121*σ213⟩ - 0.5Γ13*⟨σ211*σ123⟩ - 0.5Γ12*⟨σ211*σ122⟩ - 1.5Γ11*⟨σ221⟩
∂ₜ(⟨σ222⟩) = (0 + 1im)*g2*⟨a′*σ122⟩ + (0 - 1im)*g2*⟨a*σ212⟩ - 0.5Γ21*⟨σ121*σ212⟩ - 0.5Γ32*⟨σ122*σ213⟩ - 0.5Γ23*⟨σ212*σ123⟩ - 0.5Γ12*⟨σ211*σ122⟩ - 1.5Γ22*⟨σ222⟩
∂ₜ(⟨σ223⟩) = (0 + 1im)*g3*⟨a′*σ123⟩ + (0 - 1im)*g3*⟨a*σ213⟩ - 0.5Γ13*⟨σ211*σ123⟩ - 0.5Γ23*⟨σ212*σ123⟩ - 0.5Γ31*⟨σ121*σ213⟩ - 0.5Γ32*⟨σ122*σ213⟩ - 1.5Γ33*⟨σ223⟩
∂ₜ(⟨a′*σ121⟩) = (0.0 - 100.0im)*⟨a′*σ121⟩ + (0 + 2im)*g1*(⟨a⟩*⟨a′*σ221⟩ + ⟨a′⟩*⟨a*σ221⟩ + ⟨σ221⟩*⟨a′*a⟩ - 2⟨a′⟩*⟨a⟩*⟨σ221⟩) + (0 + 1im)*g1*⟨σ221⟩ + (0 - 1im)*g1*⟨a′*a⟩ + Γ12*⟨σ221⟩*⟨a′*σ122⟩ + Γ12*⟨σ122⟩*⟨a′*σ221⟩ + Γ13*⟨σ123⟩*⟨a′*σ221⟩ + (0 + 1im)*g2*⟨σ121*σ212⟩ + Γ13*⟨σ221⟩*⟨a′*σ123⟩ + (0 + 1im)*g3*⟨σ121*σ213⟩ + Γ12*⟨a′⟩*⟨σ221*σ122⟩ + Γ13*⟨a′⟩*⟨σ221*σ123⟩ - 0.5var"ΓIndex(ℋ(field) ⊗ ℋ(a), :k, 3)1"*⟨a′*σ121⟩ - 0.5var"ΓIndex(ℋ(field) ⊗ ℋ(a), :k, 3)3"*⟨a′*σ123⟩ - 0.5var"ΓIndex(ℋ(field) ⊗ ℋ(a), :k, 3)2"*⟨a′*σ122⟩ - 2Γ

In [104]:
typeof(arguments(arguments(me_comp_[1].rhs)[1])[2])

Sym{Parameter, DoubleNumberedVariable}

In [105]:
length(me_comp_)

33

In [ ]:
#Still need to do insertValues into...

In [106]:
# Build an ODESystem out of the MeanfieldEquations
@named sys = ODESystem(me_comp_)

Model sys with 33 equations
States (33):
  var"⟨σ221⟩"(t)
  var"⟨σ222⟩"(t)
  var"⟨σ223⟩"(t)
  var"⟨a′*σ121⟩"(t)
  var"⟨a′*σ122⟩"(t)
  var"⟨a′*σ123⟩"(t)
⋮
Parameters (0):

In [107]:
# initial state
u0 = zeros(ComplexF64, length(me_comp_))

tend = 10.0

prob = ODEProblem(sys,u0,(0.0,tend))
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

LoadError: UndefVarError: g1 not defined

In [ ]:
N = 3
ha2 = [NLevelSpace(:a,2) for i = 1:N]
hf2 = FockSpace(:field)
h2 = tensor(hf2,ha2...)

a2 = Destroy(h2,:a)
σ2(i,j,k) = Transition(h2,Symbol(:σ,k),i,j,k+1)

@cnumbers g1 g2 g3

g = [g1,g2,g3]
k = 2

H2 = -Δ*a2'*a2 + sum(g[i]*(a2'*σ2(1,2,i) + a2*σ2(2,1,i)) for i = 1:N)

(g1*(a′*σ112)+g1*(a*σ121)+g2*(a′*σ212)+g2*(a*σ221)+g3*(a′*σ312)+g3*(a*σ321)+-100.0*(a′*a))

In [ ]:
ops2 = [σ2(2,2,2)]
J2 = [σ2(1,2,1),σ2(1,2,2),σ2(1,2,3)]
rates2 = [1.1 0.45 0.3
        0.34 1.0 0.37
        0.6 0.5 0.9]
J2dagger = adjoint.(J2)

3-element Vector{Transition{ProductSpace{Vector{ConcreteHilbertSpace}}, Symbol, Int64, Int64, Nothing}}:
 σ121
 σ221
 σ321

In [ ]:
_master_lindblad(ops2[1],J2,J2dagger,rates2)

(-0.17*(σ222)+-0.17*(σ222))